1. null, 이상치 제거 (z-score 그래프 보고 3, 4, 5, 6, 7) 다양한 시도 해봐야 함
2. 스케일링 standard, minmax, robust, maxabs + log 스케일링 다 시도해 보아야 함
3. 모델은 여러개 쓰지 않더라도 랜덤, xgboost, lightGBM, gradiant 시도해 보고 -> 스태킹 앙상블 사용해보기
4. 하이퍼 파라미터 조정하기
5. 생각나면 추가적으로

In [1]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
%matplotlib inline

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, f1_score
import xgboost as xgb
from sklearn.feature_selection import RFECV

In [2]:
df = pd.read_csv('./game_result/merged_FBREF.csv')

In [3]:
df.head()

,Date,Team,Opponent,Referee,TY,OY,TR,OR,B365TW,B365D,...,Clean Sheet,Pass Completion %,Assists,Exp. Assisted Goals,Expected Assists,Tackles Won,% of Dribblers Tackled,Blocks,Interceptions,Error
0,2018-08-10,Leicester City,Manchester Utd,A Marriner,1,2,0,0,7.50,3.9,...,0,79.4,0,0.8,1.0,11,21.4,8,12,0
1,2018-08-10,Manchester Utd,Leicester City,A Marriner,2,1,0,0,1.57,3.9,...,0,80.1,1,0.7,0.3,13,50.0,15,17,0
2,2018-08-11,Bournemouth,Cardiff City,K Friend,1,1,0,0,1.90,3.6,...,1,79.0,2,1.3,1.1,5,47.1,11,4,0
3,2018-08-11,Brighton & Hove Albion,Watford,J Moss,2,2,0,0,3.40,3.2,...,0,74.4,0,0.3,0.2,11,50.0,14,17,0
4,2018-08-11,Cardiff City,Bournemouth,K Friend,1,1,0,0,4.50,3.6,...,0,58.4,0,1.5,1.2,12,47.6,16,15,0


In [4]:
df.columns

Index(['Date', 'Team', 'Opponent', 'Referee', 'TY', 'OY', 'TR', 'OR', 'B365TW',
       'B365D', 'B365OW', 'Season', 'Time', 'Round', 'Round.1', 'Table', 'Day',
       'Venue', 'Result', 'Possesion', 'Aerial Duels(%)', 'GF', 'GA',
       'Oppo. Table', 'Shot on Target', 'Shot on Target(%)', 'Goals per Shot',
       'Expected Goals', 'Save%', 'Clean Sheet', 'Pass Completion %',
       'Assists', 'Exp. Assisted Goals', 'Expected Assists', 'Tackles Won',
       '% of Dribblers Tackled', 'Blocks', 'Interceptions', 'Error'],
      dtype='object')

In [5]:
df.isnull().sum()

Date                      0
Team                      0
Opponent                  0
Referee                   0
TY                        0
OY                        0
TR                        0
OR                        0
B365TW                    0
B365D                     0
B365OW                    0
Season                    0
Time                      0
Round                     0
Round.1                   0
Table                     0
Day                       0
Venue                     0
Result                    0
Possesion                 0
Aerial Duels(%)           0
GF                        0
GA                        0
Oppo. Table               0
Shot on Target            0
Shot on Target(%)         2
Goals per Shot            2
Expected Goals            0
Save%                     0
Clean Sheet               0
Pass Completion %         0
Assists                   0
Exp. Assisted Goals       0
Expected Assists          0
Tackles Won               0
% of Dribblers Tackl

In [6]:
# 위의 null은 슈팅 자체가 0개여서 생김 -> 그러므로 0으로 대체함

In [7]:
df = df.fillna(0)

In [8]:
df.isnull().sum()

Date                      0
Team                      0
Opponent                  0
Referee                   0
TY                        0
OY                        0
TR                        0
OR                        0
B365TW                    0
B365D                     0
B365OW                    0
Season                    0
Time                      0
Round                     0
Round.1                   0
Table                     0
Day                       0
Venue                     0
Result                    0
Possesion                 0
Aerial Duels(%)           0
GF                        0
GA                        0
Oppo. Table               0
Shot on Target            0
Shot on Target(%)         0
Goals per Shot            0
Expected Goals            0
Save%                     0
Clean Sheet               0
Pass Completion %         0
Assists                   0
Exp. Assisted Goals       0
Expected Assists          0
Tackles Won               0
% of Dribblers Tackl

In [9]:
df.columns

Index(['Date', 'Team', 'Opponent', 'Referee', 'TY', 'OY', 'TR', 'OR', 'B365TW',
       'B365D', 'B365OW', 'Season', 'Time', 'Round', 'Round.1', 'Table', 'Day',
       'Venue', 'Result', 'Possesion', 'Aerial Duels(%)', 'GF', 'GA',
       'Oppo. Table', 'Shot on Target', 'Shot on Target(%)', 'Goals per Shot',
       'Expected Goals', 'Save%', 'Clean Sheet', 'Pass Completion %',
       'Assists', 'Exp. Assisted Goals', 'Expected Assists', 'Tackles Won',
       '% of Dribblers Tackled', 'Blocks', 'Interceptions', 'Error'],
      dtype='object')

In [10]:
# 지금 이 데이터셋에는 감독이 없는데 club_games 데이터셋의 감독을 넣어서 모델링
# -> club_games, games dataset 가져와야 함

In [11]:
club_games_df = pd.read_csv('./data/club_games.csv')

In [12]:
club_games_df

,game_id,club_id,own_goals,own_position,own_manager_name,opponent_id,opponent_goals,opponent_position,opponent_manager_name,hosting,is_win
0,2320450,1468,0,NaN,Holger Bachthaler,24,2,NaN,Armin Veh,Home,0
1,2320460,1,3,NaN,Jürgen Luginger,86,1,NaN,Robin Dutt,Home,1
2,2320472,2036,4,NaN,Frank Schmidt,72,5,NaN,Alexander Schmidt,Home,0
3,2321044,16,2,1.0,Jürgen Klopp,23,1,15.0,Torsten Lieberknecht,Home,1
4,2321060,23,0,18.0,Torsten Lieberknecht,24,2,11.0,Armin Veh,Home,0
...,...,...,...,...,...,...,...,...,...,...,...
134837,3142950,681,0,NaN,Imanol Alguacil,150,0,NaN,Quique Setién,Away,0
134838,3148382,150,1,NaN,Quique Setién,714,1,NaN,Rubi,Away,0
134839,3153158,418,1,NaN,Santiago Solari,131,1,NaN,Ernesto Valverde,Away,0
134840,3307741,36661,1,NaN,Aleksandr Gorshkov,32218,2,NaN,Viktor Antikhovich,Away,0


In [13]:
club_games_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 134842 entries, 0 to 134841
Data columns (total 11 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   game_id                134842 non-null  int64  
 1   club_id                134842 non-null  int64  
 2   own_goals              134842 non-null  int64  
 3   own_position           94898 non-null   float64
 4   own_manager_name       133350 non-null  object 
 5   opponent_id            134842 non-null  int64  
 6   opponent_goals         134842 non-null  int64  
 7   opponent_position      94898 non-null   float64
 8   opponent_manager_name  133350 non-null  object 
 9   hosting                134842 non-null  object 
 10  is_win                 134842 non-null  int64  
dtypes: float64(2), int64(6), object(3)
memory usage: 11.3+ MB


In [14]:
games_df = pd.read_csv('./data/games.csv')

In [15]:
games_df

,game_id,competition_id,season,round,date,home_club_id,away_club_id,home_club_goals,away_club_goals,home_club_position,...,stadium,attendance,referee,url,home_club_formation,away_club_formation,home_club_name,away_club_name,aggregate,competition_type
0,2321044,L1,2013,2. Matchday,2013-08-18,16,23,2,1,1.0,...,SIGNAL IDUNA PARK,80200.0,Peter Sippel,https://www.transfermarkt.co.uk/borussia-dortm...,4-2-3-1,4-3-2-1,Borussia Dortmund,Eintracht Braunschweig,2:1,domestic_league
1,2321060,L1,2013,3. Matchday,2013-08-25,23,24,0,2,18.0,...,EINTRACHT-Stadion,23325.0,Wolfgang Stark,https://www.transfermarkt.co.uk/eintracht-brau...,4-3-2-1,4-2-3-1,Eintracht Braunschweig,Eintracht Frankfurt Fußball AG,0:2,domestic_league
2,2321086,L1,2013,6. Matchday,2013-09-21,4,16,1,1,15.0,...,Max-Morlock-Stadion,50000.0,Knut Kircher,https://www.transfermarkt.co.uk/1-fc-nuremberg...,4-2-3-1,4-2-3-1,1.FC Nuremberg,Borussia Dortmund,1:1,domestic_league
3,2321152,L1,2013,11. Matchday,2013-11-02,44,33,0,2,7.0,...,Olympiastadion Berlin,69277.0,Günter Perl,https://www.transfermarkt.co.uk/hertha-bsc_fc-...,4-2-3-1,4-2-3-1,Hertha BSC,FC Schalke 04,0:2,domestic_league
4,2321205,L1,2013,17. Matchday,2013-12-21,41,39,2,3,14.0,...,Volksparkstadion,50000.0,Bastian Dankert,https://www.transfermarkt.co.uk/hamburger-sv_1...,4-2-3-1,4-4-2 Diamond,Hamburger SV,1. Fußball- und Sportverein Mainz 05,2:3,domestic_league
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67416,2629035,NLP,2015,Third Round,2015-10-27,383,16866,6,0,NaN,...,Philips Stadion,31000.0,Allard Lindhout,https://www.transfermarkt.co.uk/spielbericht/i...,4-3-3 Attacking,4-4-2,Eindhovense Voetbalvereniging Philips Sport Ve...,NaN,6:0,domestic_cup
67417,3092951,CLQ,2018,Qualifying Round 2nd leg,2018-08-29,383,713,3,0,NaN,...,Philips Stadion,34200.0,Anthony Taylor,https://www.transfermarkt.co.uk/spielbericht/i...,4-3-3 Defending,4-5-1,Eindhovense Voetbalvereniging Philips Sport Ve...,NaN,3:0,international_cup
67418,4164101,CGB,2023,Second Round,2023-08-29,512,1194,6,1,NaN,...,bet365 Stadium,9410.0,Michael Salisbury,https://www.transfermarkt.co.uk/spielbericht/i...,Starting Line-up: 4-2-3-1,Starting Line-up: 4-2-3-1,Stoke City,NaN,6:1,other
67419,3143618,FAC,2018,Third Round Replay,2019-01-15,512,3054,2,3,NaN,...,bet365 Stadium,10261.0,Geoff Eltringham,https://www.transfermarkt.co.uk/spielbericht/i...,4-4-2 Diamond,4-2-3-1,Stoke City,NaN,2:3,domestic_cup


In [16]:
games_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67421 entries, 0 to 67420
Data columns (total 23 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   game_id                 67421 non-null  int64  
 1   competition_id          67421 non-null  object 
 2   season                  67421 non-null  int64  
 3   round                   67421 non-null  object 
 4   date                    67421 non-null  object 
 5   home_club_id            67421 non-null  int64  
 6   away_club_id            67421 non-null  int64  
 7   home_club_goals         67421 non-null  int64  
 8   away_club_goals         67421 non-null  int64  
 9   home_club_position      47449 non-null  float64
 10  away_club_position      47449 non-null  float64
 11  home_club_manager_name  66675 non-null  object 
 12  away_club_manager_name  66675 non-null  object 
 13  stadium                 67207 non-null  object 
 14  attendance              57718 non-null

In [17]:
games_df = games_df[games_df['competition_id'] == 'GB1']

In [18]:
games_df

,game_id,competition_id,season,round,date,home_club_id,away_club_id,home_club_goals,away_club_goals,home_club_position,...,stadium,attendance,referee,url,home_club_formation,away_club_formation,home_club_name,away_club_name,aggregate,competition_type
24,2332021,GB1,2013,17. Matchday,2013-12-21,984,3008,1,1,16.0,...,The Hawthorns,24753.0,Jonathan Moss,https://www.transfermarkt.co.uk/west-bromwich-...,4-2-3-1,5-3-2,West Bromwich Albion,Hull City,1:1,domestic_league
25,2332050,GB1,2013,14. Matchday,2013-12-04,289,631,3,4,20.0,...,Stadium of Light,40652.0,Phil Dowd,https://www.transfermarkt.co.uk/sunderland-afc...,4-1-4-1,4-2-3-1,Sunderland AFC,Chelsea Football Club,3:4,domestic_league
26,2332053,GB1,2013,13. Matchday,2013-11-30,379,931,3,0,15.0,...,Boleyn Ground,34946.0,Martin Atkinson,https://www.transfermarkt.co.uk/west-ham-unite...,4-2-3-1,4-4-1-1,West Ham United Football Club,Fulham Football Club,3:0,domestic_league
27,2332188,GB1,2013,23. Matchday,2014-01-28,180,11,2,2,9.0,...,St Mary's Stadium,31284.0,Lee Mason,https://www.transfermarkt.co.uk/southampton-fc...,4-2-3-1,4-2-3-1,Southampton FC,Arsenal Football Club,2:2,domestic_league
28,2332235,GB1,2013,23. Matchday,2014-01-28,873,3008,1,0,14.0,...,Selhurst Park,22519.0,Roger East,https://www.transfermarkt.co.uk/crystal-palace...,4-4-2 double 6,3-5-2 flat,Crystal Palace Football Club,Hull City,1:0,domestic_league
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67061,2258968,GB1,2012,21. Matchday,2013-01-01,281,512,3,0,2.0,...,Etihad Stadium,47192.0,Michael Oliver,https://www.transfermarkt.co.uk/manchester-cit...,NaN,NaN,Manchester City Football Club,Stoke City,3:0,domestic_league
67152,3429694,GB1,2020,21. Matchday,2021-01-31,1237,148,1,0,17.0,...,AMEX Stadium,NaN,Peter Bankes,https://www.transfermarkt.co.uk/brighton-amp-h...,3-4-2-1,3-4-2-1,Brighton and Hove Albion Football Club,Tottenham Hotspur Football Club,1:0,domestic_league
67160,3592240,GB1,2021,37. Matchday,2022-05-15,148,1132,1,0,4.0,...,Tottenham Hotspur Stadium,61729.0,Kevin Friend,https://www.transfermarkt.co.uk/tottenham-hots...,5-4-1,5-3-2,Tottenham Hotspur Football Club,Burnley Football Club,1:0,domestic_league
67291,2872297,GB1,2017,19. Matchday,2017-12-23,1132,148,0,3,7.0,...,Turf Moor,21650.0,Michael Oliver,https://www.transfermarkt.co.uk/burnley-fc_tot...,4-2-3-1,4-2-3-1,Burnley Football Club,Tottenham Hotspur Football Club,0:3,domestic_league


In [19]:
games_df['date'] = pd.to_datetime(games_df['date'])

In [20]:
games_df = games_df[games_df['date'] >= '2018-08-01']

In [21]:
games_df

,game_id,competition_id,season,round,date,home_club_id,away_club_id,home_club_goals,away_club_goals,home_club_position,...,stadium,attendance,referee,url,home_club_formation,away_club_formation,home_club_name,away_club_name,aggregate,competition_type
1876,3050170,GB1,2018,1. Matchday,2018-08-11,1110,631,0,3,19.0,...,John Smith's Stadium,24121.0,Chris Kavanagh,https://www.transfermarkt.co.uk/huddersfield-t...,3-5-2 flat,4-3-3 Attacking,Huddersfield Town,Chelsea Football Club,0:3,domestic_league
1877,3050215,GB1,2018,5. Matchday,2018-09-15,1010,985,1,2,4.0,...,Vicarage Road,20537.0,Mike Dean,https://www.transfermarkt.co.uk/watford-fc_man...,4-4-2,4-2-3-1,Watford FC,Manchester United Football Club,1:2,domestic_league
1878,3050250,GB1,2018,9. Matchday,2018-10-20,631,985,2,2,3.0,...,Stamford Bridge,40721.0,Mike Dean,https://www.transfermarkt.co.uk/chelsea-fc_man...,4-3-3 Attacking,4-3-3 Attacking,Chelsea Football Club,Manchester United Football Club,2:2,domestic_league
1879,3050264,GB1,2018,10. Matchday,2018-10-27,180,762,0,0,16.0,...,St Mary's Stadium,30736.0,Chris Kavanagh,https://www.transfermarkt.co.uk/southampton-fc...,4-4-2 double 6,4-4-2 double 6,Southampton FC,Newcastle United Football Club,0:0,domestic_league
1880,3050336,GB1,2018,17. Matchday,2018-12-15,543,989,2,0,7.0,...,Molineux Stadium,30997.0,Simon Hooper,https://www.transfermarkt.co.uk/wolverhampton-...,3-5-2,3-4-2-1,Wolverhampton Wanderers Football Club,Association Football Club Bournemouth,2:0,domestic_league
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66901,3050278,GB1,2018,12. Matchday,2018-11-10,603,1237,2,1,18.0,...,Cardiff City Stadium,29402.0,Martin Atkinson,https://www.transfermarkt.co.uk/cardiff-city_b...,4-3-3 Attacking,4-2-3-1,Cardiff City,Brighton and Hove Albion Football Club,2:1,domestic_league
66913,3421741,GB1,2020,7. Matchday,2020-11-01,148,1237,2,1,3.0,...,Tottenham Hotspur Stadium,NaN,Graham Scott,https://www.transfermarkt.co.uk/tottenham-hots...,4-2-3-1,3-5-2 flat,Tottenham Hotspur Football Club,Brighton and Hove Albion Football Club,2:1,domestic_league
66914,3429714,GB1,2020,23. Matchday,2021-02-06,1132,1237,1,1,17.0,...,Turf Moor,NaN,Anthony Taylor,https://www.transfermarkt.co.uk/burnley-fc_bri...,4-4-2 double 6,5-3-2,Burnley Football Club,Brighton and Hove Albion Football Club,1:1,domestic_league
67152,3429694,GB1,2020,21. Matchday,2021-01-31,1237,148,1,0,17.0,...,AMEX Stadium,NaN,Peter Bankes,https://www.transfermarkt.co.uk/brighton-amp-h...,3-4-2-1,3-4-2-1,Brighton and Hove Albion Football Club,Tottenham Hotspur Football Club,1:0,domestic_league


In [22]:
games_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2183 entries, 1876 to 67160
Data columns (total 23 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   game_id                 2183 non-null   int64         
 1   competition_id          2183 non-null   object        
 2   season                  2183 non-null   int64         
 3   round                   2183 non-null   object        
 4   date                    2183 non-null   datetime64[ns]
 5   home_club_id            2183 non-null   int64         
 6   away_club_id            2183 non-null   int64         
 7   home_club_goals         2183 non-null   int64         
 8   away_club_goals         2183 non-null   int64         
 9   home_club_position      2183 non-null   float64       
 10  away_club_position      2183 non-null   float64       
 11  home_club_manager_name  2183 non-null   object        
 12  away_club_manager_name  2183 non-null   object   

In [23]:
games_df.isnull().sum()

game_id                     0
competition_id              0
season                      0
round                       0
date                        0
home_club_id                0
away_club_id                0
home_club_goals             0
away_club_goals             0
home_club_position          0
away_club_position          0
home_club_manager_name      0
away_club_manager_name      0
stadium                     0
attendance                455
referee                     0
url                         0
home_club_formation         0
away_club_formation         0
home_club_name              0
away_club_name              0
aggregate                   0
competition_type            0
dtype: int64

In [24]:
club_games_df = club_games_df[club_games_df['game_id'].isin(games_df['game_id'])]

In [25]:
club_games_df

,game_id,club_id,own_goals,own_position,own_manager_name,opponent_id,opponent_goals,opponent_position,opponent_manager_name,hosting,is_win
2097,3050170,1110,0,19.0,David Wagner,631,3,2.0,Maurizio Sarri,Home,0
2098,3050190,1110,0,19.0,David Wagner,603,0,15.0,Neil Warnock,Home,0
2099,3050199,603,2,16.0,Neil Warnock,11,3,9.0,Unai Emery,Home,0
2100,3050215,1010,1,4.0,Javi Gracia,985,2,8.0,José Mourinho,Home,0
2101,3050250,631,2,3.0,Maurizio Sarri,985,2,10.0,José Mourinho,Home,0
...,...,...,...,...,...,...,...,...,...,...,...
134815,3429732,350,0,20.0,Chris Wilder,379,3,5.0,David Moyes,Away,0
134816,3429776,399,2,11.0,Marcelo Bielsa,931,1,18.0,Scott Parker,Away,1
134818,3592178,31,2,2.0,Jürgen Klopp,405,1,15.0,Steven Gerrard,Away,1
134821,4095319,1148,2,14.0,Thomas Frank,543,0,11.0,Gary O'Neil,Away,1


In [26]:
merged_df1 = pd.merge(games_df[['game_id', 'date']], club_games_df, on='game_id', how='right')

In [27]:
merged_df1.sort_values('game_id')

,game_id,date,club_id,own_goals,own_position,own_manager_name,opponent_id,opponent_goals,opponent_position,opponent_manager_name,hosting,is_win
917,3050167,2018-08-12,11,0,15.0,Unai Emery,281,2,3.0,Pep Guardiola,Home,0
3100,3050167,2018-08-12,281,2,3.0,Pep Guardiola,11,0,15.0,Unai Emery,Away,1
4192,3050168,2018-08-11,603,0,17.0,Neil Warnock,989,2,5.0,Eddie Howe,Away,0
2009,3050168,2018-08-11,989,2,5.0,Eddie Howe,603,0,17.0,Neil Warnock,Home,1
918,3050169,2018-08-11,931,0,18.0,Slavisa Jokanovic,873,2,4.0,Roy Hodgson,Home,0
...,...,...,...,...,...,...,...,...,...,...,...,...
1156,4095365,2024-03-16,931,3,12.0,Marco Silva,148,0,5.0,Ange Postecoglou,Home,1
225,4095366,2024-03-16,1031,1,17.0,Rob Edwards,703,1,18.0,Nuno Espírito Santo,Home,0
2408,4095366,2024-03-16,703,1,18.0,Nuno Espírito Santo,1031,1,17.0,Rob Edwards,Away,0
4187,4095368,2024-03-17,405,1,4.0,Unai Emery,379,1,7.0,David Moyes,Away,0


In [28]:
merged_home_df = merged_df1[merged_df1['hosting'] == 'Home']

In [29]:
merged_away_df = merged_df1[merged_df1['hosting'] == 'Away']

In [30]:
home_game_df = pd.merge(merged_home_df, games_df[['game_id', 'home_club_name']], on='game_id')

In [31]:
home_game_df.sort_values('game_id')

,game_id,date,club_id,own_goals,own_position,own_manager_name,opponent_id,opponent_goals,opponent_position,opponent_manager_name,hosting,is_win,home_club_name
917,3050167,2018-08-12,11,0,15.0,Unai Emery,281,2,3.0,Pep Guardiola,Home,0,Arsenal Football Club
2009,3050168,2018-08-11,989,2,5.0,Eddie Howe,603,0,17.0,Neil Warnock,Home,1,Association Football Club Bournemouth
918,3050169,2018-08-11,931,0,18.0,Slavisa Jokanovic,873,2,4.0,Roy Hodgson,Home,0,Fulham Football Club
0,3050170,2018-08-11,1110,0,19.0,David Wagner,631,3,2.0,Maurizio Sarri,Home,0,Huddersfield Town
1323,3050171,2018-08-12,31,4,1.0,Jürgen Klopp,379,0,20.0,Manuel Pellegrini,Home,1,Liverpool Football Club
...,...,...,...,...,...,...,...,...,...,...,...,...,...
263,4095359,2024-03-09,543,2,9.0,Gary O'Neil,931,1,12.0,Marco Silva,Home,1,Wolverhampton Wanderers Football Club
1604,4095362,2024-03-16,1132,2,19.0,Vincent Kompany,1148,1,15.0,Thomas Frank,Home,1,Burnley Football Club
1156,4095365,2024-03-16,931,3,12.0,Marco Silva,148,0,5.0,Ange Postecoglou,Home,1,Fulham Football Club
225,4095366,2024-03-16,1031,1,17.0,Rob Edwards,703,1,18.0,Nuno Espírito Santo,Home,0,Luton Town Football Club


In [32]:
new_column_names = {'home_club_name': 'club_name'}
home_game_df = home_game_df.rename(columns=new_column_names)

In [33]:
away_game_df = pd.merge(merged_away_df, games_df[['game_id', 'away_club_name']], on='game_id')

In [34]:
away_game_df.sort_values('game_id')

,game_id,date,club_id,own_goals,own_position,own_manager_name,opponent_id,opponent_goals,opponent_position,opponent_manager_name,hosting,is_win,away_club_name
917,3050167,2018-08-12,281,2,3.0,Pep Guardiola,11,0,15.0,Unai Emery,Away,1,Manchester City Football Club
2009,3050168,2018-08-11,603,0,17.0,Neil Warnock,989,2,5.0,Eddie Howe,Away,0,Cardiff City
918,3050169,2018-08-11,873,2,4.0,Roy Hodgson,931,0,18.0,Slavisa Jokanovic,Away,1,Crystal Palace Football Club
0,3050170,2018-08-11,631,3,2.0,Maurizio Sarri,1110,0,19.0,David Wagner,Away,1,Chelsea Football Club
1323,3050171,2018-08-12,379,0,20.0,Manuel Pellegrini,31,4,1.0,Jürgen Klopp,Away,0,West Ham United Football Club
...,...,...,...,...,...,...,...,...,...,...,...,...,...
263,4095359,2024-03-09,931,1,12.0,Marco Silva,543,2,9.0,Gary O'Neil,Away,0,Fulham Football Club
1604,4095362,2024-03-16,1148,1,15.0,Thomas Frank,1132,2,19.0,Vincent Kompany,Away,0,Brentford Football Club
1156,4095365,2024-03-16,148,0,5.0,Ange Postecoglou,931,3,12.0,Marco Silva,Away,0,Tottenham Hotspur Football Club
225,4095366,2024-03-16,703,1,18.0,Nuno Espírito Santo,1031,1,17.0,Rob Edwards,Away,0,Nottingham Forest Football Club


In [35]:
new_column_names = {'away_club_name': 'club_name'}
away_game_df = away_game_df.rename(columns=new_column_names)

In [36]:
m_merged_df = pd.concat([home_game_df, away_game_df], axis=0).sort_values('game_id')

In [37]:
# 이 df가 games, club_games 합친 df
m_merged_df

,game_id,date,club_id,own_goals,own_position,own_manager_name,opponent_id,opponent_goals,opponent_position,opponent_manager_name,hosting,is_win,club_name
917,3050167,2018-08-12,11,0,15.0,Unai Emery,281,2,3.0,Pep Guardiola,Home,0,Arsenal Football Club
917,3050167,2018-08-12,281,2,3.0,Pep Guardiola,11,0,15.0,Unai Emery,Away,1,Manchester City Football Club
2009,3050168,2018-08-11,603,0,17.0,Neil Warnock,989,2,5.0,Eddie Howe,Away,0,Cardiff City
2009,3050168,2018-08-11,989,2,5.0,Eddie Howe,603,0,17.0,Neil Warnock,Home,1,Association Football Club Bournemouth
918,3050169,2018-08-11,931,0,18.0,Slavisa Jokanovic,873,2,4.0,Roy Hodgson,Home,0,Fulham Football Club
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1156,4095365,2024-03-16,931,3,12.0,Marco Silva,148,0,5.0,Ange Postecoglou,Home,1,Fulham Football Club
225,4095366,2024-03-16,1031,1,17.0,Rob Edwards,703,1,18.0,Nuno Espírito Santo,Home,0,Luton Town Football Club
225,4095366,2024-03-16,703,1,18.0,Nuno Espírito Santo,1031,1,17.0,Rob Edwards,Away,0,Nottingham Forest Football Club
2004,4095368,2024-03-17,405,1,4.0,Unai Emery,379,1,7.0,David Moyes,Away,0,Aston Villa Football Club


In [38]:
dropped_df = df.drop(['Season', 'Time', 'Round', 'Round.1', 'Day'], axis=1)

In [39]:
dropped_df.columns

Index(['Date', 'Team', 'Opponent', 'Referee', 'TY', 'OY', 'TR', 'OR', 'B365TW',
       'B365D', 'B365OW', 'Table', 'Venue', 'Result', 'Possesion',
       'Aerial Duels(%)', 'GF', 'GA', 'Oppo. Table', 'Shot on Target',
       'Shot on Target(%)', 'Goals per Shot', 'Expected Goals', 'Save%',
       'Clean Sheet', 'Pass Completion %', 'Assists', 'Exp. Assisted Goals',
       'Expected Assists', 'Tackles Won', '% of Dribblers Tackled', 'Blocks',
       'Interceptions', 'Error'],
      dtype='object')

In [40]:
# df 와 merge하기 위해서 club이름을 다시 설정

In [41]:
name_mapping = {
    'Arsenal Football Club': 'Arsenal', 
    'Manchester City Football Club': 'Manchester City',
    'Cardiff City': 'Cardiff City', 
    'Association Football Club Bournemouth': 'Bournemouth',
    'Fulham Football Club': 'Fulham', 
    'Crystal Palace Football Club': 'Crystal Palace',
    'Huddersfield Town': 'Huddersfield', 
    'Chelsea Football Club': 'Chelsea',
    'West Ham United Football Club': 'West Ham', 
    'Liverpool Football Club': 'Liverpool',
    'Manchester United Football Club': 'Manchester Utd', 
    'Leicester City': 'Leicester City',
    'Newcastle United Football Club': 'Newcastle Utd',
    'Tottenham Hotspur Football Club': 'Tottenham', 
    'Burnley Football Club': 'Burnley',
    'Southampton FC': 'Southampton', 
    'Watford FC': 'Watford',
    'Brighton and Hove Albion Football Club': 'Brighton & Hove Albion',
    'Wolverhampton Wanderers Football Club': 'Wolverhampton', 
    'Everton Football Club': 'Everton',
    'Norwich City': 'Norwich City', 
    'Sheffield United Football Club': 'Sheffield Utd',
    'Aston Villa Football Club': 'Aston Villa', 
    'Leeds United': 'Leeds United',
    'West Bromwich Albion': 'West Bromwich Albion', 
    'Brentford Football Club': 'Brentford',
    'Nottingham Forest Football Club': 'Nottingham Forest', 
    'Luton Town Football Club': 'Luton Town'
}

In [42]:
m_merged_df['club_name'] = m_merged_df['club_name'].replace(name_mapping)

In [43]:
m_merged_df

,game_id,date,club_id,own_goals,own_position,own_manager_name,opponent_id,opponent_goals,opponent_position,opponent_manager_name,hosting,is_win,club_name
917,3050167,2018-08-12,11,0,15.0,Unai Emery,281,2,3.0,Pep Guardiola,Home,0,Arsenal
917,3050167,2018-08-12,281,2,3.0,Pep Guardiola,11,0,15.0,Unai Emery,Away,1,Manchester City
2009,3050168,2018-08-11,603,0,17.0,Neil Warnock,989,2,5.0,Eddie Howe,Away,0,Cardiff City
2009,3050168,2018-08-11,989,2,5.0,Eddie Howe,603,0,17.0,Neil Warnock,Home,1,Bournemouth
918,3050169,2018-08-11,931,0,18.0,Slavisa Jokanovic,873,2,4.0,Roy Hodgson,Home,0,Fulham
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1156,4095365,2024-03-16,931,3,12.0,Marco Silva,148,0,5.0,Ange Postecoglou,Home,1,Fulham
225,4095366,2024-03-16,1031,1,17.0,Rob Edwards,703,1,18.0,Nuno Espírito Santo,Home,0,Luton Town
225,4095366,2024-03-16,703,1,18.0,Nuno Espírito Santo,1031,1,17.0,Rob Edwards,Away,0,Nottingham Forest
2004,4095368,2024-03-17,405,1,4.0,Unai Emery,379,1,7.0,David Moyes,Away,0,Aston Villa


In [44]:
# 내가 지금까지 club_games랑 games df를 전처리한 건 df DataFrame에 감독이름을 추가하기 위해서

In [45]:
# 다음은 df dataframe 정리하기

In [46]:
# 배팅 정보는 확률로 변경

In [47]:
dropped_df['B365TW'] = (1/dropped_df['B365TW'])*100

In [48]:
dropped_df['B365D'] = (1/dropped_df['B365D'])*100

In [49]:
dropped_df['B365OW'] = (1/dropped_df['B365OW'])*100

In [50]:
dropped_df.columns

Index(['Date', 'Team', 'Opponent', 'Referee', 'TY', 'OY', 'TR', 'OR', 'B365TW',
       'B365D', 'B365OW', 'Table', 'Venue', 'Result', 'Possesion',
       'Aerial Duels(%)', 'GF', 'GA', 'Oppo. Table', 'Shot on Target',
       'Shot on Target(%)', 'Goals per Shot', 'Expected Goals', 'Save%',
       'Clean Sheet', 'Pass Completion %', 'Assists', 'Exp. Assisted Goals',
       'Expected Assists', 'Tackles Won', '% of Dribblers Tackled', 'Blocks',
       'Interceptions', 'Error'],
      dtype='object')

In [51]:
dropped_df.head()

,Date,Team,Opponent,Referee,TY,OY,TR,OR,B365TW,B365D,...,Clean Sheet,Pass Completion %,Assists,Exp. Assisted Goals,Expected Assists,Tackles Won,% of Dribblers Tackled,Blocks,Interceptions,Error
0,2018-08-10,Leicester City,Manchester Utd,A Marriner,1,2,0,0,13.333333,25.641026,...,0,79.4,0,0.8,1.0,11,21.4,8,12,0
1,2018-08-10,Manchester Utd,Leicester City,A Marriner,2,1,0,0,63.694268,25.641026,...,0,80.1,1,0.7,0.3,13,50.0,15,17,0
2,2018-08-11,Bournemouth,Cardiff City,K Friend,1,1,0,0,52.631579,27.777778,...,1,79.0,2,1.3,1.1,5,47.1,11,4,0
3,2018-08-11,Brighton & Hove Albion,Watford,J Moss,2,2,0,0,29.411765,31.250000,...,0,74.4,0,0.3,0.2,11,50.0,14,17,0
4,2018-08-11,Cardiff City,Bournemouth,K Friend,1,1,0,0,22.222222,27.777778,...,0,58.4,0,1.5,1.2,12,47.6,16,15,0


In [52]:
dropped_df['Date'] = pd.to_datetime(dropped_df['Date'])

In [53]:
master_df = pd.merge(m_merged_df[['game_id', 'date', 'club_name', 'club_id', 'opponent_id', 'own_manager_name', 'opponent_manager_name']], dropped_df, \
         left_on=['date', 'club_name'], right_on=['Date', 'Team'])

In [54]:
master_df = master_df.drop(['date', 'club_name'], axis=1)

In [55]:
master_df.columns

Index(['game_id', 'club_id', 'opponent_id', 'own_manager_name',
       'opponent_manager_name', 'Date', 'Team', 'Opponent', 'Referee', 'TY',
       'OY', 'TR', 'OR', 'B365TW', 'B365D', 'B365OW', 'Table', 'Venue',
       'Result', 'Possesion', 'Aerial Duels(%)', 'GF', 'GA', 'Oppo. Table',
       'Shot on Target', 'Shot on Target(%)', 'Goals per Shot',
       'Expected Goals', 'Save%', 'Clean Sheet', 'Pass Completion %',
       'Assists', 'Exp. Assisted Goals', 'Expected Assists', 'Tackles Won',
       '% of Dribblers Tackled', 'Blocks', 'Interceptions', 'Error'],
      dtype='object')

In [56]:
new_column_order = ['game_id', 'Date', 'Team', 'Opponent', 'club_id', 'opponent_id', 'own_manager_name', 'opponent_manager_name',
       'Referee', 'TY', 'OY', 'TR', 'OR', 'B365TW', 'B365D',
       'B365OW', 'Table', 'Venue', 'Result', 'Possesion', 'Aerial Duels(%)',
       'GF', 'GA', 'Oppo. Table', 'Shot on Target', 'Shot on Target(%)',
       'Goals per Shot', 'Expected Goals', 'Save%', 'Clean Sheet',
       'Pass Completion %', 'Assists', 'Exp. Assisted Goals',
       'Expected Assists', 'Tackles Won', '% of Dribblers Tackled', 'Blocks',
       'Interceptions', 'Error']

In [57]:
master_df

,game_id,club_id,opponent_id,own_manager_name,opponent_manager_name,Date,Team,Opponent,Referee,TY,...,Clean Sheet,Pass Completion %,Assists,Exp. Assisted Goals,Expected Assists,Tackles Won,% of Dribblers Tackled,Blocks,Interceptions,Error
0,3050167,11,281,Unai Emery,Pep Guardiola,2018-08-12,Arsenal,Manchester City,M Oliver,2,...,0,77.1,0,0.5,0.6,14,40.0,13,12,1
1,3050167,281,11,Pep Guardiola,Unai Emery,2018-08-12,Manchester City,Arsenal,M Oliver,2,...,1,81.9,2,0.5,0.4,5,28.6,14,12,1
2,3050168,603,989,Neil Warnock,Eddie Howe,2018-08-11,Cardiff City,Bournemouth,K Friend,1,...,0,58.4,0,1.5,1.2,12,47.6,16,15,0
3,3050168,989,603,Eddie Howe,Neil Warnock,2018-08-11,Bournemouth,Cardiff City,K Friend,1,...,1,79.0,2,1.3,1.1,5,47.1,11,4,0
4,3050169,931,873,Slavisa Jokanovic,Roy Hodgson,2018-08-11,Fulham,Crystal Palace,M Dean,1,...,0,85.2,0,0.5,0.7,19,42.9,5,15,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4361,4095365,931,148,Marco Silva,Ange Postecoglou,2024-03-16,Fulham,Tottenham,R Jones,2,...,1,75.8,2,1.5,1.3,9,53.6,15,13,0
4362,4095366,1031,703,Rob Edwards,Nuno Espírito Santo,2024-03-16,Luton Town,Nottingham Forest,D England,1,...,0,78.2,1,0.4,0.7,5,36.4,15,5,0
4363,4095366,703,1031,Nuno Espírito Santo,Rob Edwards,2024-03-16,Nottingham Forest,Luton Town,D England,2,...,0,68.8,1,1.7,1.4,17,48.1,14,7,0
4364,4095368,405,379,Unai Emery,David Moyes,2024-03-17,Aston Villa,West Ham,J Gillett,3,...,0,85.3,1,0.5,0.9,10,53.8,9,3,1


In [58]:
master_df = master_df[new_column_order]

In [59]:
master_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4366 entries, 0 to 4365
Data columns (total 39 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   game_id                 4366 non-null   int64         
 1   Date                    4366 non-null   datetime64[ns]
 2   Team                    4366 non-null   object        
 3   Opponent                4366 non-null   object        
 4   club_id                 4366 non-null   int64         
 5   opponent_id             4366 non-null   int64         
 6   own_manager_name        4366 non-null   object        
 7   opponent_manager_name   4366 non-null   object        
 8   Referee                 4366 non-null   object        
 9   TY                      4366 non-null   int64         
 10  OY                      4366 non-null   int64         
 11  TR                      4366 non-null   int64         
 12  OR                      4366 non-null   int64   

In [60]:
# master_df 에 더해서 

In [61]:
club_value_by_game = pd.read_csv('./data/mean_value_by_game.csv')

In [62]:
master_df.head()

,game_id,Date,Team,Opponent,club_id,opponent_id,own_manager_name,opponent_manager_name,Referee,TY,...,Clean Sheet,Pass Completion %,Assists,Exp. Assisted Goals,Expected Assists,Tackles Won,% of Dribblers Tackled,Blocks,Interceptions,Error
0,3050167,2018-08-12,Arsenal,Manchester City,11,281,Unai Emery,Pep Guardiola,M Oliver,2,...,0,77.1,0,0.5,0.6,14,40.0,13,12,1
1,3050167,2018-08-12,Manchester City,Arsenal,281,11,Pep Guardiola,Unai Emery,M Oliver,2,...,1,81.9,2,0.5,0.4,5,28.6,14,12,1
2,3050168,2018-08-11,Cardiff City,Bournemouth,603,989,Neil Warnock,Eddie Howe,K Friend,1,...,0,58.4,0,1.5,1.2,12,47.6,16,15,0
3,3050168,2018-08-11,Bournemouth,Cardiff City,989,603,Eddie Howe,Neil Warnock,K Friend,1,...,1,79.0,2,1.3,1.1,5,47.1,11,4,0
4,3050169,2018-08-11,Fulham,Crystal Palace,931,873,Slavisa Jokanovic,Roy Hodgson,M Dean,1,...,0,85.2,0,0.5,0.7,19,42.9,5,15,1


In [63]:
club_value_by_game.head()

,game_id,player_club_id,market_value_in_eur
0,3050167,11,3.064286e+07
1,3050167,281,6.357143e+07
2,3050168,603,2.196429e+06
3,3050168,989,6.035714e+06
4,3050169,873,1.117857e+07


In [64]:
master_df = pd.merge(master_df, club_value_by_game, left_on=['game_id', 'club_id'], right_on=['game_id', 'player_club_id'])

In [65]:
master_df

,game_id,Date,Team,Opponent,club_id,opponent_id,own_manager_name,opponent_manager_name,Referee,TY,...,Assists,Exp. Assisted Goals,Expected Assists,Tackles Won,% of Dribblers Tackled,Blocks,Interceptions,Error,player_club_id,market_value_in_eur
0,3050167,2018-08-12,Arsenal,Manchester City,11,281,Unai Emery,Pep Guardiola,M Oliver,2,...,0,0.5,0.6,14,40.0,13,12,1,11,3.064286e+07
1,3050167,2018-08-12,Manchester City,Arsenal,281,11,Pep Guardiola,Unai Emery,M Oliver,2,...,2,0.5,0.4,5,28.6,14,12,1,281,6.357143e+07
2,3050168,2018-08-11,Cardiff City,Bournemouth,603,989,Neil Warnock,Eddie Howe,K Friend,1,...,0,1.5,1.2,12,47.6,16,15,0,603,2.196429e+06
3,3050168,2018-08-11,Bournemouth,Cardiff City,989,603,Eddie Howe,Neil Warnock,K Friend,1,...,2,1.3,1.1,5,47.1,11,4,0,989,6.035714e+06
4,3050169,2018-08-11,Fulham,Crystal Palace,931,873,Slavisa Jokanovic,Roy Hodgson,M Dean,1,...,0,0.5,0.7,19,42.9,5,15,1,931,1.060714e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4361,4095365,2024-03-16,Fulham,Tottenham,931,148,Marco Silva,Ange Postecoglou,R Jones,2,...,2,1.5,1.3,9,53.6,15,13,0,931,1.681250e+07
4362,4095366,2024-03-16,Luton Town,Nottingham Forest,1031,703,Rob Edwards,Nuno Espírito Santo,D England,1,...,1,0.4,0.7,5,36.4,15,5,0,1031,5.200000e+06
4363,4095366,2024-03-16,Nottingham Forest,Luton Town,703,1031,Nuno Espírito Santo,Rob Edwards,D England,2,...,1,1.7,1.4,17,48.1,14,7,0,703,1.485714e+07
4364,4095368,2024-03-17,Aston Villa,West Ham,405,379,Unai Emery,David Moyes,J Gillett,3,...,1,0.5,0.9,10,53.8,9,3,1,405,3.106667e+07


In [66]:
master_df.columns

Index(['game_id', 'Date', 'Team', 'Opponent', 'club_id', 'opponent_id',
       'own_manager_name', 'opponent_manager_name', 'Referee', 'TY', 'OY',
       'TR', 'OR', 'B365TW', 'B365D', 'B365OW', 'Table', 'Venue', 'Result',
       'Possesion', 'Aerial Duels(%)', 'GF', 'GA', 'Oppo. Table',
       'Shot on Target', 'Shot on Target(%)', 'Goals per Shot',
       'Expected Goals', 'Save%', 'Clean Sheet', 'Pass Completion %',
       'Assists', 'Exp. Assisted Goals', 'Expected Assists', 'Tackles Won',
       '% of Dribblers Tackled', 'Blocks', 'Interceptions', 'Error',
       'player_club_id', 'market_value_in_eur'],
      dtype='object')

In [67]:
master_df = master_df.drop('player_club_id', axis=1)

In [68]:
master_df.columns

Index(['game_id', 'Date', 'Team', 'Opponent', 'club_id', 'opponent_id',
       'own_manager_name', 'opponent_manager_name', 'Referee', 'TY', 'OY',
       'TR', 'OR', 'B365TW', 'B365D', 'B365OW', 'Table', 'Venue', 'Result',
       'Possesion', 'Aerial Duels(%)', 'GF', 'GA', 'Oppo. Table',
       'Shot on Target', 'Shot on Target(%)', 'Goals per Shot',
       'Expected Goals', 'Save%', 'Clean Sheet', 'Pass Completion %',
       'Assists', 'Exp. Assisted Goals', 'Expected Assists', 'Tackles Won',
       '% of Dribblers Tackled', 'Blocks', 'Interceptions', 'Error',
       'market_value_in_eur'],
      dtype='object')

In [69]:
master_df.isnull().sum()

game_id                   0
Date                      0
Team                      0
Opponent                  0
club_id                   0
opponent_id               0
own_manager_name          0
opponent_manager_name     0
Referee                   0
TY                        0
OY                        0
TR                        0
OR                        0
B365TW                    0
B365D                     0
B365OW                    0
Table                     0
Venue                     0
Result                    0
Possesion                 0
Aerial Duels(%)           0
GF                        0
GA                        0
Oppo. Table               0
Shot on Target            0
Shot on Target(%)         0
Goals per Shot            0
Expected Goals            0
Save%                     0
Clean Sheet               0
Pass Completion %         0
Assists                   0
Exp. Assisted Goals       0
Expected Assists          0
Tackles Won               0
% of Dribblers Tackl

In [70]:
master_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4366 entries, 0 to 4365
Data columns (total 40 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   game_id                 4366 non-null   int64         
 1   Date                    4366 non-null   datetime64[ns]
 2   Team                    4366 non-null   object        
 3   Opponent                4366 non-null   object        
 4   club_id                 4366 non-null   int64         
 5   opponent_id             4366 non-null   int64         
 6   own_manager_name        4366 non-null   object        
 7   opponent_manager_name   4366 non-null   object        
 8   Referee                 4366 non-null   object        
 9   TY                      4366 non-null   int64         
 10  OY                      4366 non-null   int64         
 11  TR                      4366 non-null   int64         
 12  OR                      4366 non-null   int64   

In [71]:
master_df

,game_id,Date,Team,Opponent,club_id,opponent_id,own_manager_name,opponent_manager_name,Referee,TY,...,Pass Completion %,Assists,Exp. Assisted Goals,Expected Assists,Tackles Won,% of Dribblers Tackled,Blocks,Interceptions,Error,market_value_in_eur
0,3050167,2018-08-12,Arsenal,Manchester City,11,281,Unai Emery,Pep Guardiola,M Oliver,2,...,77.1,0,0.5,0.6,14,40.0,13,12,1,3.064286e+07
1,3050167,2018-08-12,Manchester City,Arsenal,281,11,Pep Guardiola,Unai Emery,M Oliver,2,...,81.9,2,0.5,0.4,5,28.6,14,12,1,6.357143e+07
2,3050168,2018-08-11,Cardiff City,Bournemouth,603,989,Neil Warnock,Eddie Howe,K Friend,1,...,58.4,0,1.5,1.2,12,47.6,16,15,0,2.196429e+06
3,3050168,2018-08-11,Bournemouth,Cardiff City,989,603,Eddie Howe,Neil Warnock,K Friend,1,...,79.0,2,1.3,1.1,5,47.1,11,4,0,6.035714e+06
4,3050169,2018-08-11,Fulham,Crystal Palace,931,873,Slavisa Jokanovic,Roy Hodgson,M Dean,1,...,85.2,0,0.5,0.7,19,42.9,5,15,1,1.060714e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4361,4095365,2024-03-16,Fulham,Tottenham,931,148,Marco Silva,Ange Postecoglou,R Jones,2,...,75.8,2,1.5,1.3,9,53.6,15,13,0,1.681250e+07
4362,4095366,2024-03-16,Luton Town,Nottingham Forest,1031,703,Rob Edwards,Nuno Espírito Santo,D England,1,...,78.2,1,0.4,0.7,5,36.4,15,5,0,5.200000e+06
4363,4095366,2024-03-16,Nottingham Forest,Luton Town,703,1031,Nuno Espírito Santo,Rob Edwards,D England,2,...,68.8,1,1.7,1.4,17,48.1,14,7,0,1.485714e+07
4364,4095368,2024-03-17,Aston Villa,West Ham,405,379,Unai Emery,David Moyes,J Gillett,3,...,85.3,1,0.5,0.9,10,53.8,9,3,1,3.106667e+07


In [72]:
master_df = master_df.sort_values(['club_id', 'game_id'])

In [73]:
len(master_df)

4366

In [74]:
# master_df.to_csv('non_rolling_mean.csv', index=False)

In [75]:
# 승률 컬럼 추가

In [76]:
club_games_df = pd.read_csv('./data/club_games.csv')

In [77]:
club_games_df = club_games_df.sort_values(by=['club_id', 'game_id'])

In [78]:
# 각 클럽의 총 게임 수 카운트 및 누적치로 추가
club_games_df['total_games'] = club_games_df.groupby('club_id')['game_id'].cumcount()

# 각 클럽의 승리한 게임 수 카운트 및 누적치로 추가
club_games_df['wins'] = club_games_df.groupby('club_id')['is_win'].cumsum()
club_games_df['wins'] = club_games_df.groupby('club_id')['wins'].shift(1)
club_games_df['wins'] = club_games_df['wins'].fillna(0)

# 승리 확률 계산 및 누적치로 추가
club_games_df['win_percentage'] = (club_games_df['wins'] / club_games_df['total_games']) * 100
club_games_df['win_percentage'] = club_games_df['win_percentage'].fillna(0)

In [79]:
# 상대전적별 총 게임 수 카운트 및 누적치로 추가
club_games_df['total_games_vs_opponent'] = club_games_df.groupby(['club_id', 'opponent_id'])['game_id'].cumcount()

# 상대전적별 총 게임 수 카운트 및 누적치로 추가
club_games_df['wins_vs_opponent'] = club_games_df.groupby(['club_id', 'opponent_id'])['is_win'].cumsum()
club_games_df['wins_vs_opponent'] = club_games_df.groupby(['club_id', 'opponent_id'])['wins_vs_opponent'].shift(1)
club_games_df['wins_vs_opponent'] = club_games_df['wins_vs_opponent'].fillna(0)

# 승리 확률 계산 및 누적치로 추가
club_games_df['win_percentage_vs_opponent'] = (club_games_df['wins_vs_opponent'] / club_games_df['total_games_vs_opponent']) * 100
club_games_df['win_percentage_vs_opponent'] = club_games_df['win_percentage_vs_opponent'].fillna(0)

In [80]:
# # 각 클럽의 총 게임 수 카운트 및 누적치로 추가
# club_games_df['total_games'] = club_games_df.groupby('club_id')['game_id'].cumcount()

# # 각 클럽의 승리한 게임 수 카운트 및 누적치로 추가
# club_games_df['wins'] = (club_games_df['is_win'] == 1).groupby(club_games_df['club_id']).cumsum() - club_games_df['is_win']

# # 승리 확률 계산 및 누적치로 추가
# club_games_df['win_percentage'] = (club_games_df['wins'] / club_games_df['total_games']) * 100

In [81]:
# # 상대전적별 총 게임 수 카운트 및 누적치로 추가
# club_games_df['total_games_vs_opponent'] = club_games_df.groupby(['club_id', 'opponent_id']).cumcount()

# # 상대전적별 승리한 게임 수 카운트 및 누적치로 추가
# club_games_df['wins_vs_opponent'] = (club_games_df['is_win'] == 1).groupby([club_games_df['club_id'], club_games_df['opponent_id']]).cumsum() - club_games_df['is_win']

# # 승리 확률 계산 및 누적치로 추가
# club_games_df['win_percentage_vs_opponent'] = (club_games_df['wins_vs_opponent'] / club_games_df['total_games_vs_opponent']) * 100

In [82]:
club_games_df

,game_id,club_id,own_goals,own_position,own_manager_name,opponent_id,opponent_goals,opponent_position,opponent_manager_name,hosting,is_win,total_games,wins,win_percentage,total_games_vs_opponent,wins_vs_opponent,win_percentage_vs_opponent
63481,2221775,1,0,NaN,Jürgen Luginger,33,5,NaN,Huub Stevens,Home,0,0,0.0,0.000000,0,0.0,0.0
1,2320460,1,3,NaN,Jürgen Luginger,86,1,NaN,Robin Dutt,Home,1,1,0.0,0.000000,0,0.0,0.0
25171,2361069,1,2,NaN,Milan Sasic,127,1,NaN,André Breitenreiter,Home,1,2,1.0,50.000000,0,0.0,0.0
8500,2370752,1,0,NaN,Milan Sasic,16,2,NaN,Jürgen Klopp,Home,0,3,2.0,66.666667,0,0.0,0.0
26030,2864716,1,1,NaN,Dirk Lottner,89,2,NaN,Jens Keller,Home,0,4,2.0,50.000000,0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108617,4192074,112752,1,NaN,Javier Ubasos,112751,2,NaN,Diego Pérez,Away,0,0,0.0,0.000000,0,0.0,0.0
120921,4192077,112753,3,NaN,Juanvi López,74219,1,NaN,Nacho Aznar,Away,1,0,0.0,0.000000,0,0.0,0.0
61802,4203999,112753,0,NaN,Juanvi López,1108,10,NaN,Luis García,Home,0,1,1.0,100.000000,0,0.0,0.0
125093,4192079,112754,1,NaN,NaN,92096,3,NaN,NaN,Away,0,0,0.0,0.000000,0,0.0,0.0


In [83]:
master_df2 = pd.merge(master_df, club_games_df[['game_id', 'club_id', 'opponent_id', 'total_games', 'wins', 'win_percentage', 'total_games_vs_opponent', 'wins_vs_opponent', 'win_percentage_vs_opponent']], on=(['game_id', 'club_id', 'opponent_id']))

In [84]:
master_df2 = master_df2.sort_values(['club_id', 'game_id'])

In [85]:
master_df2.isnull().sum()

game_id                       0
Date                          0
Team                          0
Opponent                      0
club_id                       0
opponent_id                   0
own_manager_name              0
opponent_manager_name         0
Referee                       0
TY                            0
OY                            0
TR                            0
OR                            0
B365TW                        0
B365D                         0
B365OW                        0
Table                         0
Venue                         0
Result                        0
Possesion                     0
Aerial Duels(%)               0
GF                            0
GA                            0
Oppo. Table                   0
Shot on Target                0
Shot on Target(%)             0
Goals per Shot                0
Expected Goals                0
Save%                         0
Clean Sheet                   0
Pass Completion %             0
Assists 

In [87]:
master_df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4366 entries, 0 to 4365
Data columns (total 46 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   game_id                     4366 non-null   int64         
 1   Date                        4366 non-null   datetime64[ns]
 2   Team                        4366 non-null   object        
 3   Opponent                    4366 non-null   object        
 4   club_id                     4366 non-null   int64         
 5   opponent_id                 4366 non-null   int64         
 6   own_manager_name            4366 non-null   object        
 7   opponent_manager_name       4366 non-null   object        
 8   Referee                     4366 non-null   object        
 9   TY                          4366 non-null   int64         
 10  OY                          4366 non-null   int64         
 11  TR                          4366 non-null   int64       

In [88]:
master_df2

,game_id,Date,Team,Opponent,club_id,opponent_id,own_manager_name,opponent_manager_name,Referee,TY,...,Blocks,Interceptions,Error,market_value_in_eur,total_games,wins,win_percentage,total_games_vs_opponent,wins_vs_opponent,win_percentage_vs_opponent
0,3050167,2018-08-12,Arsenal,Manchester City,11,281,Unai Emery,Pep Guardiola,M Oliver,2,...,13,12,1,3.064286e+07,317,186.0,58.675079,14,4.0,28.571429
1,3050180,2018-08-18,Arsenal,Chelsea,11,631,Unai Emery,Maurizio Sarri,M Atkinson,2,...,12,12,2,3.235714e+07,318,186.0,58.490566,15,4.0,26.666667
2,3050187,2018-08-25,Arsenal,West Ham,11,379,Unai Emery,Manuel Pellegrini,G Scott,1,...,14,6,1,3.021429e+07,319,186.0,58.307210,12,9.0,75.000000
3,3050199,2018-09-02,Arsenal,Cardiff City,11,603,Unai Emery,Neil Warnock,A Taylor,4,...,6,11,2,3.235714e+07,320,187.0,58.437500,2,2.0,100.000000
4,3050212,2018-09-15,Arsenal,Newcastle Utd,11,762,Unai Emery,Rafael Benítez,L Probert,0,...,7,7,0,3.235714e+07,321,188.0,58.566978,10,9.0,90.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4361,4095317,2024-02-10,Brighton & Hove Albion,Tottenham,1237,148,Roberto De Zerbi,Ange Postecoglou,S Barrott,3,...,20,13,0,1.996667e+07,286,96.0,33.566434,14,4.0,28.571429
4362,4095328,2024-02-18,Brighton & Hove Albion,Sheffield Utd,1237,350,Roberto De Zerbi,Chris Wilder,S Attwell,4,...,10,4,0,2.200000e+07,287,96.0,33.449477,5,0.0,0.000000
4363,4095333,2024-02-24,Brighton & Hove Albion,Everton,1237,29,Roberto De Zerbi,Sean Dyche,T Harrington,2,...,6,4,0,1.906250e+07,288,97.0,33.680556,13,4.0,30.769231
4364,4095343,2024-03-02,Brighton & Hove Albion,Fulham,1237,931,Roberto De Zerbi,Marco Silva,S Hooper,2,...,15,8,3,1.890625e+07,289,97.0,33.564014,7,0.0,0.000000


In [89]:
# master_df2.to_csv('./data/master_df.csv', index = False)